In [1]:
from pandas import Series, DataFrame

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [ ]:
df_com_booking_mapped = pd.read_pickle('../com_booking_mapped')

In [ ]:
df = df_com_booking_mapped[df_com_booking_mapped.mapped_name.str.contains('deutsche', case=False).fillna(False)]

In [ ]:
df_pvt = df.pivot_table(values=['booking_net'], index=['fiscal_quarter_id'], columns=['services_indicator'], aggfunc=sum)

In [ ]:
df_com_mapping.loc[:5]

In [ ]:
df_pvt.sum

In [ ]:
xl2 = pd.ExcelFile('../ent_mapping_list.xlsx')

In [ ]:
df_ent_mapping = xl2.parse('mapping_in_account_name')

In [ ]:
xl = pd.ExcelFile('../com_mapping_list_working.xlsx')

In [ ]:
df_com_mapping = xl.parse('mapping_in_account_name')

In [ ]:
df_com_mapping[df_com_mapping.unique_name.str.contains('VERIZON', case=False)]

In [ ]:
def check_if_mapped(c_name=None, how_many_words=1, leave_last=0, data_frame=df_com_mapping):
    c_name_split = c_name.split()
    cus_name = " ".join(c_name_split[:how_many_words])
    cus_name2 = ''
    if leave_last == 0:
        cus_name2 = cus_name
    else:
        cus_name2 = cus_name[:-leave_last]
    print cus_name2
    return data_frame[(data_frame.mapable != 'Yes') & data_frame.unique_name.str.startswith(cus_name2)]

In [ ]:
def map_whatnotmapped(find_cus=None, how_many_words=1, leave_last=0, where_to=None, data_frame=df_com_mapping, ud_rep_name=None):
    c_name_split = find_cus.split()
    cus_name = " ".join(c_name_split[:how_many_words])
    cus_name2 = ''
    if leave_last == 0:
        cus_name2 = cus_name
    else:
        cus_name2 = cus_name[:-leave_last]
    print cus_name2
    rep_cus = find_cus
    if ud_rep_name is not None:
        rep_cus = ud_rep_name
    data_frame.ix[data_frame.unique_name.str.startswith(cus_name2), 'mapped_name']=rep_cus
    data_frame.ix[data_frame.unique_name.str.startswith(cus_name2), 'where_to']=where_to
    data_frame.ix[data_frame.unique_name.str.startswith(cus_name2), 'mapable']='Yes'
    return data_frame[data_frame.unique_name.str.startswith(cus_name2)&data_frame.where_to.isnull()]
    
    

In [ ]:
def check_if_mapped2(c_name=None, how_many_words=1, leave_last=0, data_frame=df_com_mapping):
    c_name_split = c_name.split()
    cus_name = " ".join(c_name_split[:how_many_words])
    cus_name2 = ''
    if leave_last == 0:
        cus_name2 = cus_name
    else:
        cus_name2 = cus_name[:-leave_last]
    print cus_name2
    return data_frame[data_frame.unique_name.str.contains(cus_name2, case=False)]

In [ ]:
df_com_mapping.rename(columns=lambda x: x.lower(), inplace=True)

In [ ]:
def map_whatnotmapped2(find_cus=None, how_many_words=1, leave_last=0, where_to=None, data_frame=df_com_mapping, ud_rep_name=None):
    c_name_split = find_cus.split()
    cus_name = " ".join(c_name_split[:how_many_words])
    cus_name2 = ''
    if leave_last == 0:
        cus_name2 = cus_name
    else:
        cus_name2 = cus_name[:-leave_last]
    print cus_name2
    rep_cus = find_cus
    if ud_rep_name is not None:
        rep_cus = ud_rep_name
    data_frame.ix[data_frame.unique_name.str.contains(cus_name2, case=False), 'mapped_name']=rep_cus
    data_frame.ix[data_frame.unique_name.str.contains(cus_name2, case=False), 'where_to']=where_to
    data_frame.ix[data_frame.unique_name.str.contains(cus_name2, case=False), 'mapable']='Yes'
    return data_frame[data_frame.unique_name.str.contains(cus_name2, case=False)&data_frame.where_to.isnull()]
    
    

In [ ]:
df_com_mapping[df_com_mapping.unique_name.str.startswith('LEVEL')]

In [ ]:
map_whatnotmapped2('LEVEL 3 COMMUNICATIONS PVT LTD', 1, 0, 'GSP')

In [ ]:
check_if_mapped2('LEVEL 3 CARRIER LTD.', 1)

In [ ]:
def quick_check_on_mapping(param='contain'):
    if param == 'contain':
        return df_com_mapping[df_com_mapping.unique_name.str.contains('VERIZON', case=False)]
    else:
        return df_com_mapping[df_com_mapping.unique_name.str.startswith('VERIZON')]

        

In [ ]:
quick_check_on_mapping('tata intera')

In [ ]:
def map_in_unique_customers(file_name=None, sheet_name='Sheet1', df_ent_mapping=None):
    if df_ent_mapping is None:
        xl = pd.ExcelFile('../ent_mapping_list.xlsx')
        df_ent_mapping = xl.parse('mapping_in_account_name')
    xl2 = pd.ExcelFile(file_name)
    df = xl2.parse(sheet_name)
    total_rows = len(df.index)
    loop_counter = 1
    print total_rows
    for index, row in df.iterrows():
        cus_name = row['customer_name']
        moving_to = row['gaining_gtmu']
        print '%d/%d - %s' % (loop_counter, total_rows, cus_name)
        loop_counter = loop_counter+1
        loop = True
        while loop:
            print_data_frame(check_if_mapped2(cus_name, 1, data_frame=df_ent_mapping))
            can_i_proceed = raw_input('Level_1: Proceed to Map Y/N: ')
            if can_i_proceed.lower() == 'y':
                print_data_frame(map_whatnotmapped2(cus_name, 1, 0, moving_to, data_frame=df_ent_mapping))
                break
            elif can_i_proceed.lower() == 'n':
                ud_cus_name = raw_input('Give me a Text: ')
                temp_list = ud_cus_name.split()
                words = len(temp_list)
                print_data_frame(check_if_mapped2(ud_cus_name, words, data_frame=df_ent_mapping))
                can_i_proceed = raw_input('Level_2: Proceed to Map Y/N')
                if can_i_proceed.lower() == 'y':
                    print_data_frame(map_whatnotmapped2(ud_cus_name, words, 0, moving_to, ud_rep_name=cus_name, data_frame=df_ent_mapping))
                    break
                else:
                    can_i_proceed = raw_input('Level_3: Continue Startswith search Y/N: ' )
                    if can_i_proceed.lower() == 'y':
                        print_data_frame(check_if_mapped(cus_name, 1, data_frame=df_ent_mapping))
                        can_i_proceed = raw_input('Level_4: Proceed to Map Y/N: ')
                        if can_i_proceed.lower() == 'y':
                            print_data_frame(map_whatnotmapped(cus_name, 1, 0, moving_to, data_frame=df_ent_mapping))
                            break
                        else:
                            df.ix[df.customer_name == cus_name, 'remarks'] = 'CHECK'
                            break
                    else:
                        df.ix[df.customer_name == cus_name, 'remarks'] = 'CHECK'
                        break
            else:
                break
        writer = pd.ExcelWriter('../ent_mapping_list_anil.xlsx', engine='xlsxwriter')
        df_ent_mapping.to_excel(writer, sheet_name='mapping_in_account_name')
        writer.save()
        writer2 = pd.ExcelWriter(file_name, engine='xlsxwriter')
        df.to_excel(writer2, sheet_name='Sheet1')
        writer2.save()
                        
            

In [ ]:
def print_data_frame(df):
    display(df)
    #for index, rows in df.iterrows():
        #print '%s | %s | %s | %s | %s | %s '% (rows['customer_name'],rows['unique_name'], rows['mapable'], rows['mapped_name'], rows['sub_scms'],rows['remarks'])

In [28]:
from IPython.display import display, HTML

In [ ]:
map_in_unique_customers('../anil_srinivas.xlsx')

In [ ]:
for i in [2016, 2015, 2014]:
    print i

In [ ]:
df_com_2016.loc[:5]

In [19]:
xl = pd.ExcelFile('../mapped_booking_data/com_booking_mapped2016.xlsx')

In [20]:
df_com_2016 = xl.parse('raw_data')

In [29]:
pd.options.display.float_format = '${:16,.0f}'.format

In [ ]:
fill_data = ['Yes','ALSTOM INDIA LTD','', '', 'INDIA_ENT_FSI_MFG_ECOMM', 'INDIA_AREA']

In [ ]:
df_ent_mapping[df_ent_mapping.unique_name.str.startswith('AXIS').fillna(False)]

In [ ]:
fill_header = ['mapable', 'mapped_name', 'gaining_sub_scms', 'remarks', 'where_to', 'gaining_sales_level_2']

In [ ]:
df_com_2016.ix[df_com_2016.bookings_adjustments_description.str.contains('Uber', case=False).fillna(False), fill_header] = fill_data

In [ ]:
df_com_2016[df_com_2016.bookings_adjustments_description.str.contains('Uber', case=False).fillna(False)]

In [ ]:
df_com_2016[df_com_2016.mapped_name.str.contains('uber', case=False).fillna(False)]

In [ ]:
pvt_com_2016 = df_com_2016.pivot_table(values=['booking_net'], index=['mapable', 'mapped_name','where_to'],columns=['services_indicator'], aggfunc=sum, fill_value=0)

In [ ]:
pvt_gap_inbound = pvt_com_2016.query('mapable == ["Yes"] and where_to == ["GAP Inbound"]')

In [ ]:
pvt_gap_inbound.sum

In [ ]:
writer = pd.ExcelWriter('../com_booking_mapped2016_v2.xlsx', engine='xlsxwriter')

In [ ]:
df_com_2016.to_excel(writer, sheet_name='raw_data')

In [ ]:
writer.save()

In [ ]:
df_ent_mapping[df_ent_mapping.unique_name.str.contains('BANK OF BARODA').fillna(False) & (df_ent_mapping.mapable == 'No')]

In [ ]:
df_ent_mapping[df_ent_mapping.unique_name.str.contains('VERIZON', case=False).fillna(False)]

In [ ]:
df_ent_mapping[df_ent_mapping.unique_name.str.startswith('BHEL').fillna(False)]

In [ ]:
df_ent_mapping.ix[df_ent_mapping.unique_name.str.contains('BHARAT HEAV', case=False).fillna(False), fill_header] = fill_data

In [ ]:
df_ent_mapping.ix[df_ent_mapping.unique_name.str.startswith('BHEL').fillna(False), fill_header] = fill_data

In [ ]:
df_

In [ ]:
fill_data = ['Yes','BHARAT HEAVY ELECTRICALS LIMITED','', '', 'INDIA_ENT_FSI_MFG_ECOMM', 'INDIA_AREA']

In [ ]:
df_com_2016 = pd.read_excel('../mapped_booking_data/com_booking_mapped2016_v1.xlsx', sheetname='raw_data')

In [61]:
pvt_com_2016 = df_com_2016.pivot_table(values='booking_net', index=['mapped_name','customer_name','sales_order_number_detail', 'fiscal_quarter_id'], columns='services_indicator', aggfunc=sum)

In [ ]:
fill_data = ['Yes','ALSTOM INDIA LTD','', '', 'INDIA_ENT_FSI_MFG_ECOMM', 'INDIA_AREA']

In [ ]:
fill_header = ['mapable', 'mapped_name', 'gaining_sub_scms', 'remarks', 'where_to', 'gaining_sales_level_2']

In [ ]:
df_com_2016.ix[df_com_2016.bookings_adjustments_description.str.contains('Uber', case=False).fillna(False), fill_header] = fill_data

In [74]:
pvt_com_2016.query('sales_order_number_detail == [101194362, 102353071, 102352659, 102110793]'), 

(services_indicator                                                                                N  \
 mapped_name                customer_name  sales_order_number_detail fiscal_quarter_id                 
 FIDELITY BUSINESS SERVICES FIDELITY INDIA 101194362                 2016Q1             61873.100185   
                                                                     2016Q2              -798.160014   
 
 services_indicator                                                                      Y  
 mapped_name                customer_name  sales_order_number_detail fiscal_quarter_id      
 FIDELITY BUSINESS SERVICES FIDELITY INDIA 101194362                 2016Q1            NaN  
                                                                     2016Q2            NaN  ,)

In [79]:
def search_for_SO(so_nos=None, df=df_com_2016):
    pvt = df.pivot_table(values='booking_net', index=['mapped_name', 'customer_name', 'sales_order_number_detail','fiscal_quarter_id'], columns='services_indicator', aggfunc=sum)    
    #display(pvt)
    #so_text = 'sales_order_number_detail == ['+str(so_no)+']'
    display(pvt.query('sales_order_number_detail == @so_nos'))


In [80]:
search_for_SO([101194362, 102353071, 102352659, 102110793])

services_indicator                                                                                N  \
mapped_name                customer_name  sales_order_number_detail fiscal_quarter_id                 
FIDELITY BUSINESS SERVICES FIDELITY INDIA 101194362                 2016Q1             61873.100185   
                                                                    2016Q2              -798.160014   

services_indicator                                                                      Y  
mapped_name                customer_name  sales_order_number_detail fiscal_quarter_id      
FIDELITY BUSINESS SERVICES FIDELITY INDIA 101194362                 2016Q1            NaN  
                                                                    2016Q2            NaN

In [12]:
df_ent_2016 = pd.read_excel('../ent_booking_mapped2016.xlsx')

In [62]:
df_com_2016.ix[(df_com_2016.where_to != 'COMMERCIAL') & (df_com_2016.mapped_name.isnull()), 'mapped_name'] = 'Retained'

In [69]:
df_com_2016.ix[~df_com_2016.where_to.isnull()]

,id,bookings_adjustments_code,bookings_adjustments_description,bookings_adjustments_type,at_attach,business_unit,customer_name,erp_deal_id,sales_order_number_detail,fiscal_period_id,...,mapable,mapped_name,gaining_sub_scms,remarks,where_to,gaining_sales_level_2,gaining_sales_level_3,gaining_sales_level_4,gaining_sales_level_5,gaining_sales_level_6
21,2271361,DP,27-Aug-15NEW POS,POS,Sales Non-AT,SSEBU,PROCTER & GAMBLE HOME PRODUCTS LTD,NaN,304547899,201602,...,Yes,PROCTER AND GAMBLE,MM,Moving from COM,INDIA_ENT_GI,INDIA_AREA,$ nan,$ nan,$ nan,$ nan
27,2271370,DP,11-Aug-15NEW POS,POS,Sales Non-AT,TECHNICAL SUPPO,TATA COMMUNICATIONS,NaN,304200671,201601,...,Yes,TATA COMMUNICATIONS,NaN,NaN,GSP,INDIA_AREA,$ nan,$ nan,$ nan,$ nan
29,2271376,DP,01-Aug-15NEW POS,POS,Sales Non-AT,ECBU,DEUTSCHE BANK,NaN,304018956,201601,...,Yes,DEUTSCHE BANK AG,GEO_NON_NA,Moving from COM (New),INDIA_ENT_GI,INDIA_AREA,$ nan,$ nan,$ nan,$ nan
40,2271403,DP,31-Aug-15NEW POS,POS,Sales Non-AT,SRGBU,TATA INTERACTIVE SERVICES LTD,NaN,304616376,201602,...,No,TATA GROUP,NaN,NaN,GSP,NaN,$ nan,$ nan,$ nan,$ nan
45,2271415,DP,25-Aug-15NEW POS,POS,Sales Non-AT,TECHNICAL SUPPO,TATASTEEL LTD,NaN,304495178,201602,...,No,TATA GROUP,NaN,NaN,GSP,NaN,$ nan,$ nan,$ nan,$ nan
52,2271434,DP,21-Aug-15NEW POS,POS,Sales Non-AT,TECHNICAL SUPPO,HONDA MOTORCYCLE & SCOOTER INDIA PVT LTD,NaN,303948005,201601,...,Yes,HONDA MOTOR COMPANY LIMITED,GEO_NON_NA,Moving from COM,INDIA_ENT_GI,INDIA_AREA,$ nan,$ nan,$ nan,$ nan
70,2271483,CM,OFFSET,BOOKING,Sales Non-AT,UABU,MONSANTO HOLDINGS PVT LTD,15862378,14190458,201602,...,Yes,MONSANTO COMPANY,SELECT,Moving from COM,INDIA_ENT_GI,INDIA_AREA,$ nan,$ nan,$ nan,$ nan
75,2271494,UNKNOWN,NOT APPLICABLE,NaN,Sales Non-AT,TMGBU,CREDIT SUISSE SERVICES INDIA PRIVATE LTD,16220483,14221217,201603,...,Yes,CREDIT-SUISSE,MM,Moving from COM,INDIA_ENT_GI,INDIA_AREA,$ nan,$ nan,$ nan,$ nan
80,2271509,CM,OFFSET,BOOKING,Sales Non-AT,TECHNICAL SUPPO,CARGILL INDIA PVT LTD,NaN,14170161,201601,...,Yes,CARGILL INDIA PRIVATE LIMITED,GEO_NAMED,Moving from COM,INDIA_ENT_GI,INDIA_AREA,$ nan,$ nan,$ nan,$ nan
94,2271548,UNKNOWN,NOT APPLICABLE,NaN,Sales Non-AT,TMGBU,CREDIT-SUISSE,16204674,101237984,201603,...,Yes,CREDIT-SUISSE,MM,Moving from COM,INDIA_ENT_GI,INDIA_AREA,$ nan,$ nan,$ nan,$ nan


In [16]:
writer = pd.ExcelWriter('../ent_booking_mapped2016_v2.xlsx', engine='xlsxwriter')

In [17]:
df_ent_2016.to_excel(writer, sheet_name='raw_data')

In [18]:
writer.save()

In [45]:
so_pvt_df_com_2016 = df_com_2016.pivot_table(values='booking_net', index=['sales_order_number_detail', 'customer_name', 'fiscal_quarter_id'], columns=['sales_level_4','services_indicator'], aggfunc=sum, fill_value=0)

In [46]:
so_pvt_df_com_2016

sales_level_4                                                                                  INDIA_COMM_EU1_SOUTH  \
services_indicator                                                                                                N   
sales_order_number_detail customer_name                                      fiscal_quarter_id                        
-9999                     UNKNOWN                                            2016Q1               $         -10,217   
                                                                             2016Q2               $          -2,328   
                                                                             2016Q3               $            -946   
 1671                     AVERY DENNISON CORPORATION                         2016Q1               $               0   
 1672                     ZF FRIEDRICHSHAFEN AG                              2016Q1               $               0   
 1673                     CATHAY PACIFIC AIRWAYS                             2016Q1               $               0   
 1674                     PHILIP MORRIS INTERNATIONAL                        2016Q1               $               0   
 1675                     PHILIP MORRIS INTERNATIONAL                        2016Q1               $               0   
 1676                     LION RESOURCES SINGAPORE                           2016Q1               $               0   
 1677                     LOREAL CHINA                                       2016Q1               $               0   
 1678                     THE CARLYLE GROUP                                  2016Q1               $           1,012   
 1679                     THE CARLYLE GROUP                                  2016Q1               $               0   
 1680                     THE CARLYLE GROUP                                  2016Q1               $           1,012   
 1681                     ASC (AUTOMATED SYSTEM CO)                          2016Q1               $             308   
 1682                     COLGATE PALMOLIVE INC                              2016Q1               $               0   
 1683                     COLGATE PALMOLIVE INC                              2016Q1               $               0   
 1684                     SITA AIRCOM                                        2016Q1               $               0   
 1685                     MERCER CONSULTING                                  2016Q1               $               0   
 1686                     COLGATE PALMOLIVE INC                              2016Q1               $               0   
 1687                     LION RESOURCES SINGAPORE                           2016Q1               $               0   
 1688                     TRW                                                2016Q1               $           3,954   
 1689                     ASC (AUTOMATED SYSTEM CO)                          2016Q1               $               0   
 1690                     DANONE                                             2016Q1               $               0   
 1691                     SINGAPORE AIRLINES                                 2016Q1               $               0   
 1692                     LION RESOURCES SINGAPORE                           2016Q1               $               0   
 1693                     CLEARWATER CAPITAL PARTNERS                        2016Q1               $               0   
 1694                     THE CARLYLE GROUP                                  2016Q1               $           1,148   
 1695                     LION RESOURCES SINGAPORE                           2016Q1               $               0   
 1696                     ASC (AUTOMATED SYSTEM CO)                          2016Q1               $               0   
 1697                     SINGAPORE AIRLINES                                 2016Q1               $               0   
...                                                                      

In [58]:
def show_com_data_by_so_no(so_no=None,df=df_com_2016):
    pvt = df.pivot_table(values='booking_net', index=['sales_order_number_detail', 'mapped_name','customer_name'], columns=['sales_level_4','services_indicator'], aggfunc=sum, fill_value=0)
    result = pvt.query('sales_order_number_detail == [@so_no]')
    display(result)

In [72]:
show_com_data_by_so_no(102353071)

sales_level_4                                                      INDIA_COMM_EU1_SOUTH  \
services_indicator                                                                    N   
sales_order_number_detail mapped_name                customer_name                        
102353071                 FIDELITY BUSINESS SERVICES UNKNOWN          $               0   

sales_level_4                                                                         \
services_indicator                                                                 Y   
sales_order_number_detail mapped_name                customer_name                     
102353071                 FIDELITY BUSINESS SERVICES UNKNOWN       $               0   

sales_level_4                                                      INDIA_COMM_EU2_WEST  \
services_indicator                                                                   N   
sales_order_number_detail mapped_name                customer_name                       
102353071                 FIDELITY BUSINESS SERVICES UNKNOWN         $               0   

sales_level_4                                                                         \
services_indicator                                                                 Y   
sales_order_number_detail mapped_name                customer_name                     
102353071                 FIDELITY BUSINESS SERVICES UNKNOWN       $               0   

sales_level_4                                                      INDIA_COMM_EU3_NE_SAARC  \
services_indicator                                                                       N   
sales_order_number_detail mapped_name                customer_name                           
102353071                 FIDELITY BUSINESS SERVICES UNKNOWN             $         103,174   

sales_level_4                                                                         \
services_indicator                                                                 Y   
sales_order_number_detail mapped_name                customer_name                     
102353071                 FIDELITY BUSINESS SERVICES UNKNOWN       $               0   

sales_level_4                                                        INDIA_COMM_MISC  \
services_indicator                                                                 N   
sales_order_number_detail mapped_name                customer_name                     
102353071                 FIDELITY BUSINESS SERVICES UNKNOWN       $               0   

sales_level_4                                                                         
services_indicator                                                                 Y  
sales_order_number_detail mapped_name                customer_name                    
102353071                 FIDELITY BUSINESS SERVICES UNKNOWN       $               0

In [51]:
def map_by_so_no(so_no=None, c_name='Retained', df=df_com_2016):
    df_com_2016.ix[df_com_2016.sales_order_number_detail == so_no, ['mapable','mapped_name', 'gaining_sub_scms', 'remarks','where_to','gaining_sales_level_2'] = ['Yes', c_name]

,id,bookings_adjustments_code,bookings_adjustments_description,bookings_adjustments_type,at_attach,business_unit,customer_name,erp_deal_id,sales_order_number_detail,fiscal_period_id,...,mapable,mapped_name,gaining_sub_scms,remarks,where_to,gaining_sales_level_2,gaining_sales_level_3,gaining_sales_level_4,gaining_sales_level_5,gaining_sales_level_6
0,2271314,DP,15-Oct-15NEW POS,POS,Sales Non-AT,TMGBU,NETWORK BULLS TECHNOLOGIES PVTLTD,NaN,305778368,201603,...,No,NaN,NaN,NaN,NaN,NaN,$ nan,$ nan,$ nan,$ nan
1,2271315,DP,31-Aug-15NEW POS,POS,Sales Non-AT,UABU,CHANNEL SALES,NaN,304613430,201602,...,No,NaN,NaN,NaN,NaN,NaN,$ nan,$ nan,$ nan,$ nan
2,2271319,UNKNOWN,NOT APPLICABLE,NaN,Sales AT,UCIBU,DHAKA WATER SUPPLY AND SEWERAGE AUTHORITY (DWASA),16214952,101251629,201603,...,No,NaN,NaN,NaN,NaN,NaN,$ nan,$ nan,$ nan,$ nan
3,2271323,DP,23-Oct-15NEW POS,POS,Sales Non-AT,UABU,QUINNOX CONSULTANCY SERVICES LTD,NaN,306004484,201603,...,No,NaN,NaN,NaN,NaN,NaN,$ nan,$ nan,$ nan,$ nan
4,2271327,DP,10-Aug-15NEW POS,POS,Sales Non-AT,SRGBU,STATE BANK OF INDIA,NaN,304173885,201601,...,No,NaN,NaN,NaN,NaN,NaN,$ nan,$ nan,$ nan,$ nan
5,2271328,UNKNOWN,NOT APPLICABLE,NaN,Sales AT,UCIBU,HOSPIRA INDIA,16105716,101101180,201603,...,No,NaN,NaN,NaN,NaN,NaN,$ nan,$ nan,$ nan,$ nan
